In [ ]:
#requirements:
#pip install plotly
#pip install -U kaleido
#pip install shapely

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from pprint import pprint
import seaborn as sb
#import altair as alt
from multiprocessing import Pool
#alt.renderers.enable('notebook')
import folium
from folium import plugins
from folium.plugins import HeatMap
%matplotlib inline
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
#import pygeohash as pgh
import matplotlib.pyplot as plt
import plotly.express as px

plt.style.use('ggplot')
plot_options = dict(figsize=(12,6))

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression

In [3]:
df = pd.read_csv("../data/Fires.csv")

C:\Users\johno\AppData\Local\Temp\ipykernel_19568\2933279801.py:1: DtypeWarning: Columns (8,10,11,12,13,14,15,16,17,18,35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/Fires.csv")


Below columns need to either be explicitly define type after reading in or dropped from DF

In [4]:
df.columns[[8,10,11,12,13,14,15,16,17,18,35,37]] 

Index(['SOURCE_REPORTING_UNIT', 'LOCAL_FIRE_REPORT_ID', 'LOCAL_INCIDENT_ID',
       'FIRE_CODE', 'FIRE_NAME', 'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME',
       'MTBS_ID', 'MTBS_FIRE_NAME', 'COMPLEX_NAME', 'COUNTY', 'FIPS_NAME'],
      dtype='object')

In [5]:
df['DISCOVERY_DATE'] = pd.to_datetime(df['DISCOVERY_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')

In [6]:
#dropping columns that will have little to no effect on model performance or contain NaN values
drop_cols = ['OBJECTID', 'FOD_ID', 'FPA_ID','NWCG_REPORTING_UNIT_ID','NWCG_REPORTING_UNIT_NAME','SOURCE_REPORTING_UNIT','SOURCE_REPORTING_UNIT_NAME','LOCAL_FIRE_REPORT_ID','LOCAL_INCIDENT_ID','FIRE_CODE','FIRE_NAME','ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME','COMPLEX_NAME','STAT_CAUSE_CODE','FIPS_CODE', 'FIPS_NAME', 'Shape','OWNER_CODE','DISCOVERY_TIME','COUNTY','CONT_DATE','CONT_DOY','CONT_TIME','SOURCE_SYSTEM','SOURCE_SYSTEM_TYPE']

#drop_cols = ['OBJECTID', 'FOD_ID', 'FPA_ID','NWCG_REPORTING_UNIT_ID','NWCG_REPORTING_UNIT_NAME','SOURCE_REPORTING_UNIT','SOURCE_REPORTING_UNIT_NAME','LOCAL_FIRE_REPORT_ID','LOCAL_INCIDENT_ID','FIRE_CODE','FIRE_NAME','ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME','COMPLEX_NAME','DISCOVERY_DATE','STAT_CAUSE_CODE','FIPS_CODE', 'FIPS_NAME', 'Shape','OWNER_CODE','DISCOVERY_TIME','COUNTY','CONT_DATE','CONT_DOY','CONT_TIME','SOURCE_SYSTEM','SOURCE_SYSTEM_TYPE']
df = df.drop(columns=drop_cols)

In [7]:
df.head()

,NWCG_REPORTING_AGENCY,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_DESCR,STATE
0,FS,2005,2005-02-02,33,Miscellaneous,0.10,A,40.036944,-121.005833,USFS,CA
1,FS,2004,2004-05-12,133,Lightning,0.25,A,38.933056,-120.404444,USFS,CA
2,FS,2004,2004-05-31,152,Debris Burning,0.10,A,38.984167,-120.735556,STATE OR PRIVATE,CA
3,FS,2004,2004-06-28,180,Lightning,0.10,A,38.559167,-119.913333,USFS,CA
4,FS,2004,2004-06-28,180,Lightning,0.10,A,38.559167,-119.933056,USFS,CA


Looking at all fires based on lat and long

In [8]:
def yearlyFires():
    years = list(df.FIRE_YEAR.unique())
    years.sort()
    for year in years:
        yearDf= df[df["FIRE_YEAR"]==year]
        print("Creating map with ", yearDf.size," fires ...")
        fig = px.scatter_geo(yearDf,lat='LATITUDE',lon='LONGITUDE',
                          size="FIRE_SIZE",
                          color = "FIRE_SIZE_CLASS")
        title = f"US Map of Fires(Year ={year})"
        fig.update_layout(title = title,geo_scope='usa', title_x=0.5)

        #fig.show()
        fig.write_image(f"USmapByYear/{year}_USmapFires.png")
#yearlyFires()

In [9]:
print("Creating map with all fires - size: ", df.size)
fig = px.scatter_geo(df,lat='LATITUDE',lon='LONGITUDE',
                  size="FIRE_SIZE",
                  color = "FIRE_SIZE_CLASS")
title = f"US Map of Fires"
fig.update_layout(title = title,geo_scope='usa', title_x=0.5)

#fig.show()
fig.write_image(f"USmapByYear/All_USmapFires.png")

Creating map with all fires - size:  20685115


In [10]:
#https://www.fs.usda.gov/about-agency/contact-us/regional-offices
#https://www.latlong.net/
#[ top left, bottom left, bottom right top right]
region1_coordinates = [(48.998625,-117.032181),(45.424841, -116.575238), (45.548077, -114.563236),(45.458889, -114.340904),(45.702748,-113.970683),(44.355692,-112.842069),(44.501816, -112.716047),(44.446140,-112.386114),(44.536460, -112.353637) ,(44.562305,-111.868789),(44.541651,-111.463793),(44.752791,-111.382034),(44.476095, -111.052484),(45.000642, -111.054512),(44.997898,-104.039652),(45.945135, -104.045442),(45.935240, -96.563685),(49.000315,-97.238073)]
region2_coordinates = [(44.476095, -111.052484),(44.476095,-110.001004),(44.159549,-109.050031),(41.000701,-109.050031),(36.998911, -109.045210),(36.689620,-109.045210),(36.689620,-103.002242),(37.000070,-103.002242),(37.000070,-94.617957),(39.101825,-94.617957),(42.435240, -96.420416),(45.301939, -96.420416),(45.604551, -96.855765),(45.935240, -96.563685),(45.00064,111.0545),(44.9979,104.0397),(45.94519,104.0454),(45.000642, -111.054512),(44.997898,-104.039652),(45.945135, -104.045442),(45.935240, -96.563685)]
region3_coordinates = [(36.689620,-114.050103) ,(36.195716,-114.050103),(36.023904,-114.152134),(36.095684,-114.748691),(35.001852,-114.633024),(34.876637,-114.635455),(34.715618,-114.486139),(34.466089,-114.402030),(34.302181,-114.138016),(34.258653,-114.132248),(34.095316,-114.422722),(33.933782,-114.535904),(33.347272,-114.698987),(33.289786,-114.726255),(33.071656,-114.687490),(32.974216,-114.472487),(32.844382,-114.469185),(32.718194,-114.719968),(32.493855,-114.813498),(31.332248,-111.074822),(31.332248,-108.208456),(31.783597,-108.208456),(31.783597,-106.527977),(31.897016,-106.645840),(32.000679,-106.645840),(32.000379,-103.064510) ,(36.689620,-109.045210),(36.689620,-103.002242)]
region4_coordinates = [(45.424841, -116.575238), (45.548077, -114.563236),(45.458889, -114.340904),(45.702748,-113.970683),(44.355692,-112.842069),(44.501816, -112.716047),(44.446140,-112.386114),(44.536460, -112.353637) ,(44.562305,-111.868789),(44.541651,-111.463793),(44.752791,-111.382034),(44.476095, -111.052484),(44.476095,-110.001004),(44.159549,-109.050031),(41.000701,-109.050031),(36.998911, -109.045210),(36.689620,-109.045210),(36.689620,-114.050103) ,(36.195716,-114.050103),(36.023904,-114.152134),(36.095684,-114.748691),(35.001852,-114.633024)   ,(38.999594,-120.001012) ,(41.994873,-120.001012) ,(41.994873,-117.026266) ]
region5_coordinates = [(35.001852,-114.633024),(34.876637,-114.635455),(34.715618,-114.486139),(34.466089,-114.402030),(34.302181,-114.138016),(34.258653,-114.132248),(34.095316,-114.422722),(33.933782,-114.535904),(33.347272,-114.698987),(33.289786,-114.726255),(33.071656,-114.687490),(32.974216,-114.472487),(32.844382,-114.469185),(32.718194,-114.719968),(38.999594,-120.001012),(41.994873,-120.001012),(32.515552,-117.235491),(34.243595,-120.939332),(40.338170,-124.717891),(41.99487,-124.598449) ]     
region6_coordinates=[(48.407326,-125.003417),(48.380882,-123.134889),(48.999592,-123.320267),(48.999099,-117.032380),(41.99487,-124.598449),(41.994873,-120.001012),(41.99487,-117.0263),(45.4284,-116.5752),(48.99863,-117.0322)]
region8_coordinates = [(36.689620,-103.002242),(31.332248,-108.208456),(31.783597,-108.208456),(31.783597,-106.527977),(31.897016,-106.645840),(32.000679,-106.645840),(32.000379,-103.064510),(37.000070,-103.002242),(37.000070,-94.617957),(36.499488,-94.617874), (36.484245,-90.159248),(35.994952,-90.375636), (36.000941,-89.706940), (36.974033,-89.137409), (37.145540,-89.089624), (37.219940,-88.948239), (37.067780,-88.554268), (37.087364,-88.448597), (37.205586,-88.450610), (37.290170,-88.512708), (37.423889,-88.409754), (37.400437,-88.368572), (37.469136,-88.095988), (37.495290,-88.062357), (37.673767,-88.146777), (37.924024,-87.904758), (38.854681,-84.793704), (39.105022,-84.487693), (38.781922,-84.216453), (38.717662,-82.882168), (38.415938,-82.590182), (38.146438,-82.639409), (37.620758,-82.348588), (37.103932,-81.727357), (37.447606,-80.304973), (38.451438,-79.228965), (39.270537,-78.389899), (39.680769,-78.173912), (39.331111,-77.762201), (39.216295,-77.465616), (39.120472,-77.528761) , (38.874998,-77.042818), (38.399797,-77.312423), (37.881357,-76.252680), (37.983175,-74.876576),(36.888408,-75.589566),(35.146863,-75.172259),(31.306715,-80.882779),(27.000408,-79.896863),(24.507143,-80.555769),(28.902397,-84.904550),(28.709861,-89.345415),(28.478349,-95.187717),(25.943227,-96.910542),(26.357420,-99.119591),(29.621221,-101.513572),(28.912015,-103.138874),(30.073848,-104.830540)]#
region9_coordinates = [(37.000070,-94.617957),(39.101825,-94.617957),(42.435240, -96.420416),(45.301939, -96.420416),(45.604551, -96.855765),(45.935240, -96.563685),(36.499488,-94.617874), (36.484245,-90.159248),(35.994952,-90.375636), (36.000941,-89.706940), (36.974033,-89.137409), (37.145540,-89.089624), (37.219940,-88.948239), (37.067780,-88.554268), (37.087364,-88.448597), (37.205586,-88.450610), (37.290170,-88.512708), (37.423889,-88.409754), (37.400437,-88.368572), (37.469136,-88.095988), (37.495290,-88.062357), (37.673767,-88.146777), (37.924024,-87.904758), (38.854681,-84.793704), (39.105022,-84.487693), (38.781922,-84.216453), (38.717662,-82.882168), (38.415938,-82.590182), (38.146438,-82.639409), (37.620758,-82.348588), (37.103932,-81.727357), (37.447606,-80.304973), (38.451438,-79.228965), (39.270537,-78.389899), (39.680769,-78.173912), (39.331111,-77.762201), (39.216295,-77.465616), (39.120472,-77.528761) , (38.874998,-77.042818), (38.399797,-77.312423), (37.881357,-76.252680), (37.983175,-74.876576),(49.00032,-97.23807),(49.368066,-94.978894),(46.483265,-84.392468),(44.071800,-82.152187),(41.656497,-82.635385),(44.918139,-74.948146),(47.323931,-69.237625),(47.234490,-68.079804),(44.543505,-66.937208),(40.896906,-69.484978)]
region10_coordinates = [(70.641769,-166.837630),(65.594800,-168.271506),(60.053874,-167.876564),(57.715885,-159.793981),(57.868132,-158.483863),(52.133488,-177.723924),(53.357109,-187.651444),(55.751849,-193.449818),(55.553495,-194.679777),(54.546580,-194.416214),(51.481383,-186.333632),(50.541363,-176.230403),(59.063154,-143.548657),(54.431713,-133.649274),(55.216490,-129.871545),(59.745326,-135.401178),(58.893296,-137.377897),(60.307906,-140.988369),(69.885010,-140.718507)]

In [11]:
def createRegionDF(coords,regionName,regionDF):
    coords = [ (coord[0],coord[1],regionName) for coord in coords]
    if(len(regionDF)==0 ):
        regionDF = pd.DataFrame(data= coords, index=None, columns=["lat","lon","region"] )
    else:
        newRegionDF = pd.DataFrame(data= coords, index=None, columns=["lat","lon","region"] )
        regionDF = pd.concat([regionDF,newRegionDF],axis=0)
    return regionDF
regionDF = createRegionDF(region1_coordinates,1,pd.DataFrame())
regionDF = createRegionDF(region2_coordinates,2,regionDF)
regionDF = createRegionDF(region3_coordinates,3,regionDF)
regionDF = createRegionDF(region4_coordinates,4,regionDF)
regionDF = createRegionDF(region5_coordinates,5,regionDF)
regionDF = createRegionDF(region6_coordinates,6,regionDF)
regionDF = createRegionDF(region8_coordinates,8,regionDF)
regionDF = createRegionDF(region9_coordinates,9,regionDF)
regionDF = createRegionDF(region10_coordinates,10,regionDF)

print("Regions in Dataframe ",regionDF["region"].unique())

Regions in Dataframe  [ 1  2  3  4  5  6  8  9 10]


In [12]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
 
def checkIfCoordInRegion(regionName,regionDF,df):#reference: https://stackoverflow.com/questions/61042261/how-to-check-if-a-lat-lon-coordinate-is-within-a-polygon-using-python-taking
    polygon = Polygon([tuple(x) for x in regionDF[['lat', 'lon']].to_numpy()])
    df[f'Within_{regionName}'] = df.apply(lambda x: polygon.contains(Point(x['LATITUDE'], x['LONGITUDE'])), axis=1)
    return df


for i in range(1,11):
    if(i!=7):
        df = checkIfCoordInRegion(f"Region{i}",regionDF[regionDF["region"]==i],df)


In [13]:
region1Coord_test =df[df["STATE"]== "CA"][0:15][["LATITUDE","LONGITUDE","Within_Region1","Within_Region2","Within_Region5"]]
print(region1Coord_test)

     LATITUDE   LONGITUDE  Within_Region1  Within_Region2  Within_Region5
0   40.036944 -121.005833           False           False           False
1   38.933056 -120.404444           False           False            True
2   38.984167 -120.735556           False           False            True
3   38.559167 -119.913333           False           False            True
4   38.559167 -119.933056           False           False            True
5   38.635278 -120.103611           False           False            True
6   38.688333 -120.153333           False           False            True
7   40.968056 -122.433889           False           False           False
8   41.233611 -122.283333           False           False           False
9   38.548333 -120.149167           False           False            True
10  38.691667 -120.159722           False           False            True
11  38.527500 -120.106111           False           False            True
12  38.786667 -120.193333           Fa

Can see from the test are polygons are not 100% accurate at identifying the correct region for lat,long which is to be expected, will be handled below...

In [14]:
def regionNum(x):
    #print(x)
    if(x['STATE']=='HI'):
        return 5
    else:
        #print(x[["LATITUDE","LONGITUDE","STATE"]])
        listRegion = list(x[["Within_Region1","Within_Region2","Within_Region3","Within_Region4","Within_Region5","Within_Region6","Within_Region8","Within_Region9","Within_Region10"]])
        #print(istRegion)
        if(np.sum(listRegion)!=1):
                return -1
        if(x["Within_Region1"]):
            return 1
        if(x["Within_Region2"]):
            return 2
        if(x["Within_Region3"]):
            return 3
        if(x["Within_Region4"]):
            return 4
        if(x["Within_Region5"]):
            return 5
        if(x["Within_Region6"]):
            return 6
        if(x["Within_Region8"]):
            return 8
        if(x["Within_Region9"]):
            return 9
        if(x["Within_Region10"]):
            return 10
def reformatRegionClassification(df):
    df["Region"] = df.apply( lambda x: regionNum(x), axis=1)
    return df
df = reformatRegionClassification(df)


In [15]:
print(df[df["Region"]== -1].size)

5169423


Meaning 5169423 are not classified correctly, which means we neet either delete classify them correctly. The caveat is there are some states in which the region boundaries over lap, hence we will not use state to classify those ones.Will also exclude puerto rico from the df as it is out of scope of this project.

In [16]:
statesLeft = df[df["Region"]==-1]["STATE"].unique()
overlapStates = ['OK','WY','ID','PR']
for i in range(len(overlapStates)):
    statesLeft=statesLeft[statesLeft != overlapStates[i]]
print("States with overlap:", overlapStates)


States with overlap: ['OK', 'WY', 'ID', 'PR']


In [17]:
print(f"Before adjustments there are ",df[df["Region"]== -1].size ," rows classified incorrectly") 

#will go by state(assuming no overlap within state with region) to classify 
df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'MT') , 1, df['Region'])
df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'SD') , 2, df['Region'])
df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'KS') , 2, df['Region'])

df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'AZ') , 3, df['Region'])

df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'NM') , 4, df['Region'])
df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'NV') , 4, df['Region'])


df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'CA') , 5, df['Region'])

df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'WA') , 6, df['Region'])
df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'OR') , 6, df['Region'])

df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'TX') , 8, df['Region'])
df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'FL') , 8, df['Region'])
df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'KY') , 8, df['Region'])

df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'NJ') , 9, df['Region'])
df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'MD') , 9, df['Region'])
df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'WV') , 9, df['Region'])
df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'DE') , 9, df['Region'])
df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'PA') , 9, df['Region'])
df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'NY') , 9, df['Region'])
df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'MN') , 9, df['Region'])
df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'MO') , 9, df['Region'])
df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'IN') , 9, df['Region'])
df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'IL') , 9, df['Region'])
df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'WI') , 9, df['Region'])
df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'MI') , 9, df['Region'])
df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'OH') , 9, df['Region'])
df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'ME') , 9, df['Region'])
df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'IA') , 9, df['Region'])

df['Region'] = np.where((df['Region'] ==-1) & (df['STATE'] == 'AK') , 10, df['Region'])

#rest of them to a region else will be dropped from the DS
print("After adjustments: ",df[df["Region"]== -1].size, " --these rows will be dropped from the df")
#df = df[df['Region'] != -1]
print("Final count of misclassified fire regions: ",df[df["Region"]== -1].size)

Before adjustments there are  5169423  rows classified incorrectly
After adjustments:  481215  --these rows will be dropped from the df
Final count of misclassified fire regions:  481215


In [18]:
import plotly.express as px
#import plotly.graph_objects as go
import pandas as pd
#check to make sure forming polygons correctly
for i in range(1,11):
    if(i!=7):
        fig = px.scatter_geo(regionDF[regionDF["region"]==i],lat='lat',lon='lon',color = "region")
        fig.update_layout(title = f'Region {i}',geo_scope='usa', title_x=0.5)
        fig.write_image(f"Regions/region{i}.png")
        #fig.show()

![alt text](Regions/region1.png "Region1")
![alt text](Regions/region2.png "Region2")
![alt text](Regions/region3.png "Region3")
![alt text](Regions/region4.png "Region4")
![alt text](Regions/region5.png "Region5")
![alt text](Regions/region6.png "Region6")
![alt text](Regions/region8.png "Region8")
![alt text](Regions/region9.png "Region9")
![alt text](Regions/region10.png "Region10")

In [21]:
fig = px.scatter_geo(regionDF,lat='lat',lon='lon',color = "region")
fig.update_layout(title = 'US map By region borders',geo_scope='usa', title_x=0.5)
fig.write_image(f"Regions/allRegions.png")
fig.show()

In [ ]:
# separation of training and testing data 
label_names = sorted(df['STAT_CAUSE_DESCR'].unique())
df_dummies_X = df.drop(columns=['STAT_CAUSE_DESCR'])
df_dummies_X = pd.get_dummies(df_dummies_X)
X = df_dummies_X
Y = df['STAT_CAUSE_DESCR']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=1, stratify=Y)

In [20]:
#df_dummies_X = df.drop(columns=['STAT_CAUSE_DESCR'])
#df_dummies_X = pd.get_dummies(df_dummies_X)
#X = df_dummies_X
#Y = df['STAT_CAUSE_DESCR']

#X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.4, random_state=1, stratify=Y)


classifier = RandomForestClassifier(n_estimators=500, max_depth=50, min_samples_leaf=50, random_state=123)

classifier = classifier.fit(X_train, Y_train)
training_accuracy = accuracy_score(Y_train, classifier.predict(X_train))
pred = classifier.predict(X_test)
test_accuracy = accuracy_score(Y_test, pred)
print("Training accuracy:",training_accuracy)
print("Testing accuracy:",test_accuracy)

TypeError: The DType <class 'numpy.dtype[datetime64]'> could not be promoted by <class 'numpy.dtype[float64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[int64]'>, <class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[bool_]'>, <class 'numpy.dtype[bool_]'>, <class 'numpy.dtype[bool_]'>, <class 'numpy.dtype[bool_]'>, <class 'numpy.dtype[bool_]'>, <class 'numpy.dtype[bool_]'>, <class 'numpy.dtype[bool_]'>, <class 'numpy.dtype[bool_]'>, <class 'numpy.dtype[bool_]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[uint8]'>, <class 'numpy.dtype[uint8]'>, <class 'numpy.dtype[uint8]'>, <class 'numpy.dtype[uint8]'>, <class 'numpy.dtype[uint8]'>, <class 'numpy.dtype[uint8]'>, <class 'numpy.dtype[uint8]'>, <class 'numpy.d

In [ ]:
%matplotlib inline
import seaborn as sb
import matplotlib.pyplot as plt
import altair as alt
label_names = sorted(df['STAT_CAUSE_DESCR'].unique())

matrix = confusion_matrix(Y_test, pred)
plt.figure(figsize=(8,8))
sb.heatmap(matrix,square=True,annot=True, fmt='d', cbar=False, cmap='Blues',
          xticklabels=label_names,yticklabels=label_names)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.savefig("randomforestconfusionmatrix.png", dpi=170)